<a href="https://colab.research.google.com/github/darkzard05/Cora_dataset_benchmark/blob/main/sgconv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch-scatter torch-sparse torch-spline-conv torch_geometric -f https://data.pyg.org/whl/torch-1.11.0+cu113.html

Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 7.9 MB 7.0 MB/s 
     |████████████████████████████████| 3.5 MB 59.6 MB/s 
     |████████████████████████████████| 750 kB 48.5 MB/s 
     |████████████████████████████████| 407 kB 7.8 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=da9fdee35e4a3eb9c4779a492be408a4f0088b31aa2237c41e35b722b575f106
  Stored in directory: /root/.cache/pip/wheels/18/a6/a4/ca18c3051fcead866fe7b85700ee2240d883562a1bc70ce421
Successfully built torch-geometric


In [2]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.nn import Linear
import torch_geometric
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import SGConv

print(torch.__version__)

1.11.0+cu113


In [3]:
dataset_name = 'Cora'
dataset = Planetoid(root='/tmp/'+dataset_name, name=dataset_name, 
                    transform=T.TargetIndegree()
                    )

Processing...
Done!


In [4]:
data = dataset[0]
print(f'dataset name: {dataset_name}')
print(f'number of nodes: {data.num_nodes} // number of edges: {data.num_edges} // number of classes: {dataset.num_classes}')
print(f'number of node features: {data.num_node_features} // number of edge features: {data.num_edge_features}')
print(data.x, data.edge_index, data.edge_attr, data.y)
print(data.train_mask.sum(), data.val_mask.sum(), data.test_mask.sum())

dataset name: Cora
number of nodes: 2708 // number of edges: 10556 // number of classes: 7
number of node features: 1433 // number of edge features: 1
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]) tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]]) tensor([[0.0179],
        [0.0238],
        [0.0179],
        ...,
        [0.1964],
        [0.0238],
        [0.0238]]) tensor([3, 4, 4,  ..., 3, 3, 3])
tensor(140) tensor(500) tensor(1000)


In [5]:
class Model(torch.nn.Module):
    def __init__(self, K):
        super().__init__()
        self.layer1 = SGConv(data.num_node_features, dataset.num_classes, K=K)

    def reset_parameters(self):
        self.layer1.reset_parameters()

    def forward(self, d):
        x, edge_index = d.x, d.edge_index
        x = F.dropout(x, training=self.training)
        x = self.layer1(x, edge_index).relu()
        return F.log_softmax(x, dim=1)

In [6]:
print('cuda' if torch.cuda.is_available() else 'cpu', 'is available')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Model(K=2).to(device)
print(model)
model.reset_parameters()
print({i:list(j.shape) for i, j in model.named_parameters()})

data = dataset[0].to(device)
print(data)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-1, weight_decay=5e-3)

def train():
    model.train()
    optimizer.zero_grad(set_to_none=True)
    loss = torch.nn.CrossEntropyLoss()(model(data)[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss

@torch.no_grad()
def test(mask):
    model.eval()
    correct = model(data).argmax(dim=1)[mask] == data.y[mask]
    return int(correct.sum()) / int(mask.sum())

best_acc = 0
epochs = 1000
for epoch in range(1, epochs+1):
    loss = train()
    train_acc = test(data.train_mask)
    val_acc = test(data.val_mask)
    test_acc = test(data.test_mask)
    if best_acc < test_acc:
        best_acc = test_acc
        print(f'epoch: {epoch} | loss: {loss.item():.4f} | train_acc: {train_acc:.4f} | val_acc: {val_acc} | test_acc: {test_acc}')

cuda is available
Model(
  (layer1): SGConv(1433, 7, K=2)
)
{'layer1.lin.weight': [7, 1433], 'layer1.lin.bias': [7]}
Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_attr=[10556, 1])
epoch: 1 | loss: 1.9428 | train_acc: 0.6714 | val_acc: 0.472 | test_acc: 0.517
epoch: 2 | loss: 1.2083 | train_acc: 0.8429 | val_acc: 0.698 | test_acc: 0.712
epoch: 4 | loss: 0.4161 | train_acc: 0.9643 | val_acc: 0.702 | test_acc: 0.723
epoch: 5 | loss: 0.3621 | train_acc: 0.9929 | val_acc: 0.734 | test_acc: 0.783
epoch: 6 | loss: 0.2623 | train_acc: 1.0000 | val_acc: 0.78 | test_acc: 0.814
epoch: 7 | loss: 0.2340 | train_acc: 1.0000 | val_acc: 0.796 | test_acc: 0.815
epoch: 16 | loss: 0.3956 | train_acc: 0.9929 | val_acc: 0.78 | test_acc: 0.818
epoch: 17 | loss: 0.3966 | train_acc: 1.0000 | val_acc: 0.778 | test_acc: 0.822
epoch: 29 | loss: 0.3802 | train_acc: 1.0000 | val_acc: 0.778 | test_acc: 0.831
epoch: 30 | loss: 0.3653 | train_acc: 1.0